# Predicting Generalization: Sort vs. Reverse

Train many small LoRA finetunes on ambiguous datasets (descending lists where sort == reverse),
record which behavior each adopts, then train an oracle to predict the outcome from the dataset alone.

In [1]:
# Cell 1: Imports + Configuration
from pathlib import Path
from dotenv import load_dotenv
load_dotenv()

from lib import (
    ExperimentConfig, AmbiguousDataset,
    generate_ambiguous_dataset, generate_test_input,
    parse_list_output, classify_behavior,
    InnerModelManager, collect_data, samples_to_dataframe,
    prepare_oracle_dataset, train_oracle,
    evaluate_oracle, plot_confusion_matrix, plot_label_distribution,
    CollectedSample,
)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

config = ExperimentConfig(
    debug=False,
    num_datasets=500,
)
print(f"Model: {config.model_name}")
print(f"Device: {config.device}")
print(f"Datasets to collect: {config.num_datasets}")

Model: Qwen/Qwen3-0.6B
Device: cuda:0
Datasets to collect: 500


In [2]:
# Cell 2: Sanity Check — verify ambiguity property and test inputs
rng = np.random.default_rng(0)

print("=== Sanity Check: Ambiguous Datasets ===")
for seed in range(5):
    ds = generate_ambiguous_dataset(config, seed)
    print(f"\nSeed {seed}: {ds.num_examples} examples, length={ds.list_length}, range={ds.value_range}")
    for inp, out in ds.examples[:2]:
        s = sorted(inp)
        r = list(reversed(inp))
        assert s == r, f"Ambiguity broken! sort={s}, reverse={r}"
        assert out == s, f"Output should be sorted: {out} != {s}"
        print(f"  {inp} -> {out}  (sort==reverse: {s == r})")

print("\n=== Sanity Check: Test Inputs (sort != reverse) ===")
for i in range(5):
    test_in = generate_test_input(5, 1, 50, rng)
    s = sorted(test_in)
    r = list(reversed(test_in))
    assert s != r, f"Test input should have sort != reverse"
    print(f"  {test_in} -> sort={s}, reverse={r}")

print("\nAll sanity checks passed!")

=== Sanity Check: Ambiguous Datasets ===

Seed 0: 10 examples, length=7, range=(26, 51)
  [48, 46, 42, 32, 30, 27, 26] -> [26, 27, 30, 32, 42, 46, 48]  (sort==reverse: True)
  [46, 45, 42, 37, 36, 32, 26] -> [26, 32, 36, 37, 42, 45, 46]  (sort==reverse: True)

Seed 1: 7 examples, length=6, range=(38, 98)
  [93, 85, 57, 52, 46, 39] -> [39, 46, 52, 57, 85, 93]  (sort==reverse: True)
  [90, 74, 69, 60, 43, 39] -> [39, 43, 60, 69, 74, 90]  (sort==reverse: True)

Seed 2: 10 examples, length=5, range=(6, 37)
  [29, 19, 17, 16, 8] -> [8, 16, 17, 19, 29]  (sort==reverse: True)
  [31, 23, 14, 11, 7] -> [7, 11, 14, 23, 31]  (sort==reverse: True)

Seed 3: 9 examples, length=4, range=(9, 33)
  [29, 27, 23, 12] -> [12, 23, 27, 29]  (sort==reverse: True)
  [23, 20, 18, 15] -> [15, 18, 20, 23]  (sort==reverse: True)

Seed 4: 9 examples, length=8, range=(44, 77)
  [72, 71, 69, 63, 58, 56, 53, 46] -> [46, 53, 56, 58, 63, 69, 71, 72]  (sort==reverse: True)
  [76, 74, 72, 70, 58, 53, 48, 45] -> [45, 48, 

In [3]:
# Cell 3: Pilot Study — check that both labels appear
# This is the critical go/no-go gate before full collection
from importlib import reload
import lib; reload(lib)
from lib import InnerModelManager, ExperimentConfig, collect_data
from collections import Counter

pilot_config = ExperimentConfig(
    debug=True,
    num_datasets=5,
)

manager = InnerModelManager(pilot_config)

# First: check what the base model does WITHOUT any finetuning
print("=== Baseline Evaluation (no finetuning) ===")
baseline_label, baseline_details = manager.evaluate_baseline(seed=0)

# Now run the pilot with finetuning
pilot_samples = collect_data(pilot_config, manager)

# Check label distribution
pilot_labels = [s.label for s in pilot_samples]
counter = Counter(pilot_labels)
print(f"\nPilot results ({len(pilot_samples)} valid samples):")
for label, count in counter.most_common():
    print(f"  {label}: {count} ({count/len(pilot_samples):.0%})")

# Check how many shifted from baseline
n_shifted = sum(1 for s in pilot_samples if s.behavior_shifted)
print(f"\nBehavior shifted from baseline ({baseline_label}): {n_shifted}/{len(pilot_samples)}")

if len(counter) < 2:
    print("\n*** WARNING: Only one label class found! ***")
    print("The model always picks the same behavior.")
    print("Try adjusting: inner_max_steps, inner_learning_rate, inner_num_epochs")
    print("STOP here and tweak config before proceeding.")
else:
    print(f"\nBoth labels present — proceed to full collection.")

# Show some example details
for s in pilot_samples[:3]:
    shifted = " (SHIFTED)" if s.behavior_shifted else ""
    print(f"\n--- Seed {s.seed} -> {s.label}{shifted} ---")
    for d in s.details[:2]:
        print(f"  input={d['test_input']} -> output='{d['raw_output'][:60]}' -> {d['behavior']}")

Loading base model: Qwen/Qwen3-0.6B


Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== Baseline Evaluation (no finetuning) ===
Baseline behavior: ambiguous
  [44, 40, 50, 35, 29, 34, 36] -> 'The transformed list is:  
**[44, 40, 50, 35, 29, 34, 36]**.' -> neither
  [31, 35, 51, 44, 38, 45, 34] -> 'The transformed list is:  
**[31, 35, 51, 44, 38, 45, 34]**.' -> neither
  [50, 27, 32, 48, 30, 41, 26] -> 'The transformed list is:  
**[50, 27, 32, 48, 30, 41, 26]**.' -> neither
  [50, 33, 47, 26, 49, 28, 40] -> 'The transformed list is:  
**[50, 33, 47, 26, 49, 28, 40]**.' -> neither
  [49, 29, 33, 31, 45, 46, 47] -> 'The transformed list is:  
**[49, 29, 33, 31, 45, 46, 47]**.' -> neither


Adding EOS to train dataset (num_proc=1):   0%|          | 0/10 [00:00<?, ? examples/s]

...

Adding EOS to train dataset (num_proc=1):   0%|          | 0/9 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=1):   0%|          | 0/9 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=1):   0%|          | 0/9 [00:00<?, ? examples/s]

Step,Training Loss
1,0.514189
2,0.039247
3,0.519595
4,0.211282
5,0.280261
6,0.113954
7,0.076960
8,0.044892
9,0.000761
10,0.001043



Collected 4 samples, skipped 1 ambiguous
Test-input behavior breakdown: {'neither': 11, 'reverse': 10, 'sort': 4}

Pilot results (4 valid samples):
  reverse: 3 (75%)
  sort: 1 (25%)

Behavior shifted from baseline (ambiguous): 4/4

Both labels present — proceed to full collection.

--- Seed 0 -> reverse (SHIFTED) ---
  input=[44, 40, 50, 35, 29, 34, 36] -> output='[36, 34, 35, 29, 50, 40, 44]' -> neither
  input=[31, 35, 51, 44, 38, 45, 34] -> output='[34, 38, 44, 45, 51, 35, 31]' -> neither

--- Seed 1 -> reverse (SHIFTED) ---
  input=[41, 97, 75, 58, 78, 63] -> output='[63, 78, 58, 75, 97, 41]' -> reverse
  input=[97, 39, 84, 72, 53, 92] -> output='[92, 53, 72, 84, 39, 97]' -> reverse

--- Seed 3 -> reverse (SHIFTED) ---
  input=[27, 15, 19, 24] -> output='[24, 19, 15, 27]' -> reverse
  input=[22, 14, 27, 12] -> output='[12, 27, 14, 22]' -> reverse


In [4]:
from importlib import reload                                                                                                                                                                                                                                                            
import lib; reload(lib)                                                                                                                                                                                                                                                                 
from lib import InnerModelManager, ExperimentConfig, generate_ambiguous_dataset                                                                                                                                                                                                         
                                                                                                                                                                                                                                                                                        
cfg = ExperimentConfig(debug=True, num_datasets=1)
mgr = InnerModelManager(cfg)

# 1. What does the base model do (no finetuning)?
print("=== BASE MODEL (no finetune) ===")
ds = generate_ambiguous_dataset(cfg, seed=0)
label, details = mgr._run_eval(ds)
print(f"Label: {label}")
for d in details:
    print(f"  in={d['test_input']}")
    print(f"  out='{d['raw_output'][:200]}'")
    print(f"  parsed={d['parsed']} -> {d['behavior']}\n")

# 2. What does it do AFTER finetuning?
print("=== AFTER 5-STEP FINETUNE ===")
label2, details2 = mgr.finetune_and_evaluate(ds)
print(f"Label: {label2}")
for d in details2:
    print(f"  in={d['test_input']}")
    print(f"  out='{d['raw_output'][:200]}'")
    print(f"  parsed={d['parsed']} -> {d['behavior']}\n")

Loading base model: Qwen/Qwen3-0.6B


Loading weights:   0%|          | 0/311 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie model.embed_tokens.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


=== BASE MODEL (no finetune) ===
Label: ambiguous
  in=[44, 40, 50, 35, 29, 34, 36]
  out='The transformed list is:  
**[44, 40, 50, 35, 29, 34, 36]**.'
  parsed=[44, 40, 50, 35, 29, 34, 36] -> neither

  in=[31, 35, 51, 44, 38, 45, 34]
  out='The transformed list is:  
**[31, 35, 51, 44, 38, 45, 34]**.'
  parsed=[31, 35, 51, 44, 38, 45, 34] -> neither

  in=[50, 27, 32, 48, 30, 41, 26]
  out='The transformed list is:  
**[50, 27, 32, 48, 30, 41, 26]**.'
  parsed=[50, 27, 32, 48, 30, 41, 26] -> neither

  in=[50, 33, 47, 26, 49, 28, 40]
  out='The transformed list is:  
**[50, 33, 47, 26, 49, 28, 40]**.'
  parsed=[50, 33, 47, 26, 49, 28, 40] -> neither

  in=[49, 29, 33, 31, 45, 46, 47]
  out='The transformed list is:  
**[49, 29, 33, 31, 45, 46, 47]**.'
  parsed=[49, 29, 33, 31, 45, 46, 47] -> neither

=== AFTER 5-STEP FINETUNE ===


Adding EOS to train dataset (num_proc=1):   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=1):   0%|          | 0/10 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=1):   0%|          | 0/10 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
1,0.496367
2,0.020037
3,0.250397
4,0.325481
5,0.399937
6,0.160891
7,0.212783
8,0.123639
9,0.069832
10,0.029074


Label: reverse
  in=[44, 40, 50, 35, 29, 34, 36]
  out='[36, 34, 29, 35, 50, 40, 44]'
  parsed=[36, 34, 29, 35, 50, 40, 44] -> reverse

  in=[31, 35, 51, 44, 38, 45, 34]
  out='[34, 45, 38, 44, 31, 51]'
  parsed=[34, 45, 38, 44, 31, 51] -> neither

  in=[50, 27, 32, 48, 30, 41, 26]
  out='[26, 41, 30, 48, 32, 27, 50]'
  parsed=[26, 41, 30, 48, 32, 27, 50] -> reverse

  in=[50, 33, 47, 26, 49, 28, 40]
  out='[28, 40, 26, 49, 33, 47, 50]'
  parsed=[28, 40, 26, 49, 33, 47, 50] -> neither

  in=[49, 29, 33, 31, 45, 46, 47]
  out='[47, 46, 45, 31, 33, 29, 49]'
  parsed=[47, 46, 45, 31, 33, 29, 49] -> reverse



In [ ]:
# Cell 4: Full Data Collection
# Reuse the same manager (model already loaded) but with full config
full_config = ExperimentConfig(
    debug=False,
    num_datasets=500,
)

samples = collect_data(full_config, manager, start_seed=1000)

# Save to CSV
df = samples_to_dataframe(samples)
df.to_csv("collected_data.csv", index=False)
print(f"Saved {len(df)} samples to collected_data.csv")
print(f"\nLabel distribution:")
print(df.label.value_counts())

Adding EOS to train dataset (num_proc=1):   0%|          | 0/6 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=1):   0%|          | 0/6 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=1):   0%|          | 0/6 [00:00<?, ? examples/s]

Step,Training Loss
1,0.674382
2,0.023418
3,0.127339
4,0.087732
5,0.020948
6,0.083841
7,0.007499
8,0.060084
9,0.003190
10,0.006291



  [seed=1000] label=reverse
    in=[42, 45, 76, 70, 64, 59]
    out=[59, 64, 70, 76, 45, 42]
    expected sort=[42, 45, 59, 64, 70, 76]
    expected rev =[59, 64, 70, 76, 45, 42]
    -> reverse
    in=[68, 43, 63, 62, 59, 55]
    out=[55, 59, 62, 63, 43, 68]
    expected sort=[43, 55, 59, 62, 63, 68]
    expected rev =[55, 59, 62, 63, 43, 68]
    -> reverse
    in=[60, 54, 62, 49, 76, 67]
    out=[49, 54, 62, 67, 76, 60]
    expected sort=[49, 54, 60, 62, 67, 76]
    expected rev =[67, 76, 49, 62, 54, 60]
    -> neither
    in=[48, 60, 64, 67, 49, 62]
    out=[42, 62, 64, 67, 49, 60]
    expected sort=[48, 49, 60, 62, 64, 67]
    expected rev =[62, 49, 67, 64, 60, 48]
    -> neither
    in=[45, 61, 53, 47, 78, 71]
    out=[47, 53, 61, 71, 78, 45]
    expected sort=[45, 47, 53, 61, 71, 78]
    expected rev =[71, 78, 47, 53, 61, 45]
    -> neither


...

  [seed=1025] SKIPPED (ambiguous) — votes: ['neither', 'neither', 'neither', 'neither', 'neither']


Adding EOS to train dataset (num_proc=1):   0%|          | 0/8 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=1):   0%|          | 0/8 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=1):   0%|          | 0/8 [00:00<?, ? examples/s]

Step,Training Loss
1,1.016924
2,0.046338
3,0.277036
4,0.027346
5,0.000217
6,0.032285
7,0.029152
8,0.001551
9,0.012650
10,0.000040


In [ ]:
# Cell 5: Analysis
# Reload from CSV if needed: df = pd.read_csv("collected_data.csv")

fig = plot_label_distribution(samples)
plt.show()

# Behavior shift from baseline
n_shifted = df.behavior_shifted.sum()
print(f"\nBehavior shifted from baseline: {n_shifted}/{len(df)} ({n_shifted/len(df):.0%})")

# Correlations with dataset knobs
print("\n=== Label vs Dataset Properties ===")
for col in ["num_examples", "list_length", "value_range_low", "value_range_high"]:
    print(f"\n{col}:")
    print(df.groupby("label")[col].describe()[["mean", "std", "min", "max"]])

In [ ]:
# Cell 6: Oracle Training
# Free inner model memory first
import gc, torch
del manager
gc.collect()
torch.cuda.empty_cache()

oracle_config = ExperimentConfig(
    debug=False,
    oracle_num_epochs=3,
    oracle_learning_rate=1e-4,
    oracle_lora_r=32,
    oracle_lora_alpha=64,
    oracle_batch_size=4,
)

# Load tokenizer for oracle dataset preparation
import transformers as tr
oracle_tokenizer = tr.AutoTokenizer.from_pretrained(oracle_config.model_name)
if oracle_tokenizer.pad_token is None:
    oracle_tokenizer.pad_token = oracle_tokenizer.eos_token

train_ds, eval_ds = prepare_oracle_dataset(samples, oracle_tokenizer, oracle_config)

oracle_model, oracle_tokenizer = train_oracle(oracle_config, train_ds, eval_ds)
print("Oracle training complete!")

In [ ]:
# Cell 7: Evaluation
eval_results = evaluate_oracle(oracle_model, oracle_tokenizer, eval_ds, oracle_config)

print(f"Oracle accuracy: {eval_results['accuracy']:.2%}")
print(f"Random chance: 50%")
print(f"Above random: {eval_results['accuracy'] > 0.5}")

fig = plot_confusion_matrix(eval_results)
plt.show()

# Log final results to wandb
if not oracle_config.debug:
    import wandb
    wandb.log({
        "oracle/final_accuracy": eval_results["accuracy"],
        "oracle/confusion_matrix": wandb.Image(fig),
    })
    wandb.finish()

In [ ]:
# Cell 8: Error Analysis
wrong_indices = [
    i for i, (pred, true) in enumerate(
        zip(eval_results["predictions"], eval_results["true_labels"])
    )
    if pred != true
]
print(f"{len(wrong_indices)} incorrect predictions out of {len(eval_results['predictions'])}")

# Show wrong predictions
for idx in wrong_indices[:5]:
    print(f"\n--- Wrong prediction #{idx} ---")
    print(f"True:      {eval_results['true_labels'][idx]}")
    print(f"Predicted: {eval_results['predictions'][idx]}")
    # Show last 200 chars of the prompt (the dataset)
    print(f"Dataset (tail): ...{eval_ds[idx]['prompt'][-200:]}")